In [1]:
#Importing dependencies
import psycopg2 as ps
import pandas as pd
import gian_config as creds
import numpy as np
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
    conn = ps.connect(conn_string)
    print("Connected!")
    
    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [3]:
conn, cursor = connect()

Connected!


In [4]:
customer_df = pd.read_sql_query('SELECT * from customer_data', con=conn)
store_df = pd.read_sql_query('SELECT * from store_data', con=conn)
promo_df = pd.read_sql_query('SELECT * from promo_data', con=conn)
product_df = pd.read_sql_query('SELECT * from product_data', con=conn)
location_df = pd.read_sql_query('SELECT * from location_data', con=conn)
full_dataset = pd.read_sql_query('SELECT * from full_dataset', con=conn)

In [5]:
full_dataset

,promotion_id,promotion_name,media_type,cost,store_sales_in_millions,store_cost_in_millions,unit_sales_in_millions,product_id,store_id,customer_id,...,brand_name,food_category,food_department,food_family,srp,gross_weight,net_weight,recyclable_package,low_fat,units_per_case
0,prom0,Bag Stuffers,"Daily Paper, Radio",126.62,7.36,2.7232,4,prod0,s0,c0,...,Carrington,Breakfast Foods,Frozen Foods,Food,1.84,19.70,17.70,1,0,17
1,prom1,Cash Register Lottery,"Daily Paper, Radio",59.86,5.52,2.5944,3,prod0,s0,c1,...,Carrington,Breakfast Foods,Frozen Foods,Food,1.84,19.70,17.70,1,0,17
2,prom2,High Roller Savings,"Daily Paper, Radio",84.16,3.68,1.3616,2,prod0,s0,c2,...,Carrington,Breakfast Foods,Frozen Foods,Food,1.84,19.70,17.70,1,0,17
3,prom3,Cash Register Lottery,In-Store Coupon,95.78,3.68,1.1776,2,prod0,s0,c3,...,Carrington,Breakfast Foods,Frozen Foods,Food,1.84,19.70,17.70,1,0,17
4,prom4,Double Down Sale,Radio,50.79,4.08,1.4280,3,prod4,s0,c4,...,Golden,Breakfast Foods,Frozen Foods,Food,1.36,7.12,5.11,0,1,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38887,prom38887,Unbeatable Price Savers,"Sunday Paper, Radio",78.45,1.21,0.4477,1,prod38742,s126,c11331,...,Toretti,Specialty,Carousel,Non-Consumable,1.21,18.90,15.80,0,0,26
38888,prom38888,Price Cutters,Sunday Paper,69.42,1.60,0.4960,1,prod38744,s126,c15346,...,Symphony,Specialty,Carousel,Non-Consumable,1.60,17.40,15.30,1,0,36
38889,prom38889,You Save Days,In-Store Coupon,95.25,2.76,1.3248,1,prod38745,s126,c15344,...,ADJ,Specialty,Carousel,Non-Consumable,2.76,19.60,18.60,1,0,26
38890,prom38890,Weekend Markdown,"Sunday Paper, Radio, TV",67.51,5.52,2.5392,2,prod38745,s126,c21457,...,ADJ,Specialty,Carousel,Non-Consumable,2.76,19.60,18.60,1,0,26


In [6]:
full_dataset.columns

Index(['promotion_id', 'promotion_name', 'media_type', 'cost',
       'store_sales_in_millions', 'store_cost_in_millions',
       'unit_sales_in_millions', 'product_id', 'store_id', 'customer_id',
       'location_id', 'store_city', 'store_state', 'marital_status', 'gender',
       'total_children', 'education', 'member_card', 'occupation',
       'houseowner', 'avg_cars_at_home_approx', 'avg_yearly_income',
       'num_children_at_home', 'store_type', 'store_sqft', 'grocery_sqft',
       'frozen_sqft', 'meat_sqft', 'coffee_bar', 'video_store', 'salad_bar',
       'prepared_food', 'florist', 'brand_name', 'food_category',
       'food_department', 'food_family', 'srp', 'gross_weight', 'net_weight',
       'recyclable_package', 'low_fat', 'units_per_case'],
      dtype='object')

In [7]:
customer_df = full_dataset[['marital_status', 'gender', 'total_children', 'education','member_card', 'occupation',
                          'houseowner', 'avg_cars_at_home_approx', 'avg_yearly_income', 'num_children_at_home',
                      'promotion_name','media_type']]

customer_df

,marital_status,gender,total_children,education,member_card,occupation,houseowner,avg_cars_at_home_approx,avg_yearly_income,num_children_at_home,promotion_name,media_type
0,M,F,1,Partial High School,Normal,Skilled Manual,Y,1.0,$10K - $30K,1.0,Bag Stuffers,"Daily Paper, Radio"
1,M,M,0,Bachelors Degree,Silver,Professional,Y,4.0,$50K - $70K,0.0,Cash Register Lottery,"Daily Paper, Radio"
2,S,F,4,Partial High School,Normal,Manual,N,1.0,$10K - $30K,0.0,High Roller Savings,"Daily Paper, Radio"
3,M,F,2,High School Degree,Bronze,Manual,Y,2.0,$30K - $50K,2.0,Cash Register Lottery,In-Store Coupon
4,M,M,0,Partial High School,Bronze,Skilled Manual,N,2.0,$30K - $50K,0.0,Double Down Sale,Radio
...,...,...,...,...,...,...,...,...,...,...,...,...
38887,S,F,1,Partial High School,Bronze,Skilled Manual,N,2.0,$50K - $70K,0.0,Unbeatable Price Savers,"Sunday Paper, Radio"
38888,S,F,2,High School Degree,Bronze,Skilled Manual,N,2.0,$30K - $50K,0.0,Price Cutters,Sunday Paper
38889,M,F,1,Partial High School,Normal,Skilled Manual,Y,1.0,$10K - $30K,1.0,You Save Days,In-Store Coupon
38890,M,M,1,High School Degree,Bronze,Manual,Y,3.0,$30K - $50K,0.0,Weekend Markdown,"Sunday Paper, Radio, TV"


In [8]:
customer_df.columns

Index(['marital_status', 'gender', 'total_children', 'education',
       'member_card', 'occupation', 'houseowner', 'avg_cars_at_home_approx',
       'avg_yearly_income', 'num_children_at_home', 'promotion_name',
       'media_type'],
      dtype='object')

In [9]:
promo_df

,promotion_id,promotion_name,media_type,cost,store_sales_in_millions,store_cost_in_millions,unit_sales_in_millions,product_id,store_id,customer_id,location_id
0,prom0,Bag Stuffers,"Daily Paper, Radio",126.62,7.36,2.7232,4,prod0,s0,c0,loc0
1,prom1,Cash Register Lottery,"Daily Paper, Radio",59.86,5.52,2.5944,3,prod0,s0,c1,loc0
2,prom2,High Roller Savings,"Daily Paper, Radio",84.16,3.68,1.3616,2,prod0,s0,c2,loc0
3,prom3,Cash Register Lottery,In-Store Coupon,95.78,3.68,1.1776,2,prod0,s0,c3,loc0
4,prom4,Double Down Sale,Radio,50.79,4.08,1.4280,3,prod4,s0,c4,loc0
...,...,...,...,...,...,...,...,...,...,...,...
38887,prom38887,Unbeatable Price Savers,"Sunday Paper, Radio",78.45,1.21,0.4477,1,prod38742,s126,c11331,loc126
38888,prom38888,Price Cutters,Sunday Paper,69.42,1.60,0.4960,1,prod38744,s126,c15346,loc126
38889,prom38889,You Save Days,In-Store Coupon,95.25,2.76,1.3248,1,prod38745,s126,c15344,loc126
38890,prom38890,Weekend Markdown,"Sunday Paper, Radio, TV",67.51,5.52,2.5392,2,prod38745,s126,c21457,loc126


In [10]:
promo_df.columns

Index(['promotion_id', 'promotion_name', 'media_type', 'cost',
       'store_sales_in_millions', 'store_cost_in_millions',
       'unit_sales_in_millions', 'product_id', 'store_id', 'customer_id',
       'location_id'],
      dtype='object')

In [11]:
promo_df.drop(['promotion_id', 'store_sales_in_millions', 'store_cost_in_millions', 'unit_sales_in_millions', 'product_id', 'store_id', 'location_id', 'customer_id', 'promotion_name', 'media_type'], axis=1, inplace=True)

promo_df

,cost
0,126.62
1,59.86
2,84.16
3,95.78
4,50.79
...,...
38887,78.45
38888,69.42
38889,95.25
38890,67.51


In [12]:
customer_df = customer_df.join(promo_df)

customer_df

,marital_status,gender,total_children,education,member_card,occupation,houseowner,avg_cars_at_home_approx,avg_yearly_income,num_children_at_home,promotion_name,media_type,cost
0,M,F,1,Partial High School,Normal,Skilled Manual,Y,1.0,$10K - $30K,1.0,Bag Stuffers,"Daily Paper, Radio",126.62
1,M,M,0,Bachelors Degree,Silver,Professional,Y,4.0,$50K - $70K,0.0,Cash Register Lottery,"Daily Paper, Radio",59.86
2,S,F,4,Partial High School,Normal,Manual,N,1.0,$10K - $30K,0.0,High Roller Savings,"Daily Paper, Radio",84.16
3,M,F,2,High School Degree,Bronze,Manual,Y,2.0,$30K - $50K,2.0,Cash Register Lottery,In-Store Coupon,95.78
4,M,M,0,Partial High School,Bronze,Skilled Manual,N,2.0,$30K - $50K,0.0,Double Down Sale,Radio,50.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38887,S,F,1,Partial High School,Bronze,Skilled Manual,N,2.0,$50K - $70K,0.0,Unbeatable Price Savers,"Sunday Paper, Radio",78.45
38888,S,F,2,High School Degree,Bronze,Skilled Manual,N,2.0,$30K - $50K,0.0,Price Cutters,Sunday Paper,69.42
38889,M,F,1,Partial High School,Normal,Skilled Manual,Y,1.0,$10K - $30K,1.0,You Save Days,In-Store Coupon,95.25
38890,M,M,1,High School Degree,Bronze,Manual,Y,3.0,$30K - $50K,0.0,Weekend Markdown,"Sunday Paper, Radio, TV",67.51


In [13]:
# conn.close()
# cursor.close()

# Do different promotions work better on different:
## education levels


In [14]:
cu_edu = customer_df[[ 'education', 'promotion_name']]
cu_edu

,education,promotion_name
0,Partial High School,Bag Stuffers
1,Bachelors Degree,Cash Register Lottery
2,Partial High School,High Roller Savings
3,High School Degree,Cash Register Lottery
4,Partial High School,Double Down Sale
...,...,...
38887,Partial High School,Unbeatable Price Savers
38888,High School Degree,Price Cutters
38889,Partial High School,You Save Days
38890,High School Degree,Weekend Markdown


### Encode

In [15]:
le = LabelEncoder()

edu_enc = cu_edu.apply(le.fit_transform)

edu_enc

,education,promotion_name
0,4,0
1,0,6
2,4,18
3,2,6
4,4,11
...,...,...
38887,4,45
38888,2,24
38889,4,48
38890,2,47


### LinearRegression 

In [16]:
# Split preprocessed data into our target and features arrays
y = edu_enc['promotion_name'].values
X = edu_enc.drop(columns='promotion_name').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [17]:
# Fit the model
mlg = LinearRegression()
mlg.fit(X_train, y_train)

LinearRegression()

In [18]:
# Predict the result
y_pred = mlg.predict(X_test)
y_pred

array([25.90273882, 25.09922901, 25.90273882, ..., 25.09922901,
       25.50098392, 25.09922901])

In [19]:
# Evaluate the model
accuracy = r2_score(y_test, y_pred) 
print(f"Accuracy of multiple linear regression is {accuracy:.2f}.")

Accuracy of multiple linear regression is 0.00.


### RandomForestClassifier

In [20]:
y = edu_enc['promotion_name'].values
X = edu_enc.drop(columns='promotion_name').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# # Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=78, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.048


### Conclusion: A Customer's Education Level cannot predict sales Promotion

# Do different promotions work better on different:
## Income levels

### LinearRegression

In [22]:
cu_inc = customer_df[[ 'promotion_name', 'avg_yearly_income']]
cu_inc

,promotion_name,avg_yearly_income
0,Bag Stuffers,$10K - $30K
1,Cash Register Lottery,$50K - $70K
2,High Roller Savings,$10K - $30K
3,Cash Register Lottery,$30K - $50K
4,Double Down Sale,$30K - $50K
...,...,...
38887,Unbeatable Price Savers,$50K - $70K
38888,Price Cutters,$30K - $50K
38889,You Save Days,$10K - $30K
38890,Weekend Markdown,$30K - $50K


In [23]:
inc_enc = cu_inc.apply(le.fit_transform)

inc_enc

,promotion_name,avg_yearly_income
0,0,0
1,6,5
2,18,0
3,6,4
4,11,4
...,...,...
38887,45,5
38888,24,4
38889,48,0
38890,47,4


In [24]:
# Split preprocessed data into our features and target arrays
y = inc_enc['promotion_name'].values
X = inc_enc.drop(columns='promotion_name').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [25]:
# Fit the model
mlg = LinearRegression()
mlg.fit(X_train, y_train)

LinearRegression()

In [26]:
# Predict the result
y_pred = mlg.predict(X_test)
y_pred

array([25.35706197, 25.43444463, 25.35706197, ..., 25.74397528,
       25.27967931, 25.74397528])

In [27]:
# Evaluate the model
accuracy = r2_score(y_test, y_pred) 
print(f"Accuracy of multiple linear regression is {accuracy:.2f}.")

Accuracy of multiple linear regression is 0.00.


### RandomForestClassifier

In [28]:
y = inc_enc['promotion_name'].values
X = inc_enc.drop(columns='promotion_name').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=78, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.050


## Conclusion: Type of Promotion Cannot be predicted by Education OR Income level

- RandomForrestClassifier outperformes LinerarRegression

# Do different Media Types work better on different:
## Education Levels

In [30]:
med_edu = customer_df[[ 'media_type', 'education']]

med_edu

,media_type,education
0,"Daily Paper, Radio",Partial High School
1,"Daily Paper, Radio",Bachelors Degree
2,"Daily Paper, Radio",Partial High School
3,In-Store Coupon,High School Degree
4,Radio,Partial High School
...,...,...
38887,"Sunday Paper, Radio",Partial High School
38888,Sunday Paper,High School Degree
38889,In-Store Coupon,Partial High School
38890,"Sunday Paper, Radio, TV",High School Degree


### Encode

In [31]:
med_edu_enc = med_edu.apply(le.fit_transform)

med_edu_enc

,media_type,education
0,3,4
1,3,0
2,3,4
3,5,2
4,7,4
...,...,...
38887,10,4
38888,9,2
38889,5,4
38890,11,2


In [32]:
y = med_edu_enc['media_type'].values
X = med_edu_enc.drop(columns='media_type').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=78, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.130


# Do different Media Types work better on different:
## Income Levels

In [34]:
med_inc = customer_df[[ 'media_type', 'avg_yearly_income']]

med_inc

,media_type,avg_yearly_income
0,"Daily Paper, Radio",$10K - $30K
1,"Daily Paper, Radio",$50K - $70K
2,"Daily Paper, Radio",$10K - $30K
3,In-Store Coupon,$30K - $50K
4,Radio,$30K - $50K
...,...,...
38887,"Sunday Paper, Radio",$50K - $70K
38888,Sunday Paper,$30K - $50K
38889,In-Store Coupon,$10K - $30K
38890,"Sunday Paper, Radio, TV",$30K - $50K


In [35]:
med_inc_enc = med_inc.apply(le.fit_transform)

med_inc_enc

,media_type,avg_yearly_income
0,3,0
1,3,5
2,3,0
3,5,4
4,7,4
...,...,...
38887,10,5
38888,9,4
38889,5,0
38890,11,4


In [36]:
y = med_inc_enc['media_type'].values
X = med_inc_enc.drop(columns='media_type').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=78, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.131


### Conclusion: Though better than Promotion, Media Type is a still weakly predicted by education or income alone

# Are combined customer demographics good features for the following targets:
## - Media Type
## - Promotion Name

### Media Type

In [42]:
cu_combo = customer_df.drop(['gender', 'member_card', 'avg_cars_at_home_approx', 'total_children', 'marital_status', 'promotion_name', 'cost'], axis=1)
cu_combo

,education,occupation,houseowner,avg_yearly_income,num_children_at_home,media_type
0,Partial High School,Skilled Manual,Y,$10K - $30K,1.0,"Daily Paper, Radio"
1,Bachelors Degree,Professional,Y,$50K - $70K,0.0,"Daily Paper, Radio"
2,Partial High School,Manual,N,$10K - $30K,0.0,"Daily Paper, Radio"
3,High School Degree,Manual,Y,$30K - $50K,2.0,In-Store Coupon
4,Partial High School,Skilled Manual,N,$30K - $50K,0.0,Radio
...,...,...,...,...,...,...
38887,Partial High School,Skilled Manual,N,$50K - $70K,0.0,"Sunday Paper, Radio"
38888,High School Degree,Skilled Manual,N,$30K - $50K,0.0,Sunday Paper
38889,Partial High School,Skilled Manual,Y,$10K - $30K,1.0,In-Store Coupon
38890,High School Degree,Manual,Y,$30K - $50K,0.0,"Sunday Paper, Radio, TV"


In [43]:
combo_enc = cu_combo.apply(le.fit_transform)

combo_enc

,education,occupation,houseowner,avg_yearly_income,num_children_at_home,media_type
0,4,4,1,0,1,3
1,0,3,1,5,0,3
2,4,2,0,0,0,3
3,2,2,1,4,2,5
4,4,4,0,4,0,7
...,...,...,...,...,...,...
38887,4,4,0,5,0,10
38888,2,4,0,4,0,9
38889,4,4,1,0,1,5
38890,2,2,1,4,0,11


In [44]:
# Split preprocessed data into our features and target arrays
y = combo_enc['media_type'].values
X = combo_enc.drop(columns ='media_type').values

# X= combo_enc.drop('media_type', axis =1) 
# y= combo_enc['media_type']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [45]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.131
